# IMO Result Insights with Apache Spark

The goal of this project is to get some insights or information of the all time results of International Mathematics Olympiad (IMO) from some favorite countries (in this case I choose 15, they are: Australia, Bulgaria, Canada, China, Germany, Hungary, India, Indonesia, Iran, Japan, Korea, Romania, Russia, Thailand, United States) by doing some queries using PySpark, the Apache Spark API for Python. Firstly I will do some web scrape task to get data from the official website of IMO, and then store them as Spark dataframe.

This project is also aimed to showcase my familiarity in using data processing tool, in this case PySpark in order to solve some analytical and data engineering problems.


In [1]:
#Import some libraries needed for web scrapping (Beautifulsoup) and PySpark

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0/; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}


In [3]:
# Iniate a spark session

spark = SparkSession.builder\
            .appName('IMO_Spark')\
            .master('local')\
            .config('spark.driver.memory', '5G')\
            .getOrCreate()

In [4]:
spark

In [5]:
#Web scraping part, here the tuple of data is appended into an empty list 'contestants'
#as you can see we only care with IMO results data from these 15 countries, due to a long time needed to send request to the webpage
#The 'contestants' list later is going to be rows of spark dataframe

contestants = []
#'THA', 'IRN', 'GER', 'AUS', 'POL'
negara = ['AUS', 'BGR', 'CAN', 'CHN', 'GER', 'USA', 'JPN', 'KOR', 'ROU', 'IRN', 'IDN', 'RUS', 'HUN', 'IND', 'THA']

for c in negara:
    imo_link = f'https://www.imo-official.org/country_individual_r.aspx?code={c}'
    req_imo = requests.get(imo_link)
    print(c, 'REQUEST STATUS:', req_imo.status_code, imo_link, end=' ')

    soup = BeautifulSoup(req_imo.text)


    country_details = soup.find('h2').find_all('a')[1]
    country = country_details.getText()
    country_code = country_details['href'][-3:]

    tabel = soup.find('table').find('tbody').find_all('tr')
    for i in range(len(tabel)):
        #id_contestant = tabel[i].find_all('td')[1].find('a')['href'].split('=')[-1]
        name = tabel[i].find_all('td')[1].getText()
        year_participation = int(tabel[i].find_all('td')[0].getText())
        try:
            points = [int(tabel[i].find_all('td')[j].getText()) for j in range(2, 8)]
        except ValueError:
            points = [None for j in range(6)]
        total_points = int(tabel[i].find_all('td')[9].getText())
        abs_rank = int(tabel[i].find_all('td')[10].getText())
        rel_rank = float(tabel[i].find_all('td')[11].getText()[:-1])
        prize =  tabel[i].find_all('td')[12].getText()

        data = tuple([#id_contestant, 
                    name, country_code, country, year_participation, 
                    points[0], points[1], points[2], points[3], points[4], points[5],
                    total_points, abs_rank, rel_rank, prize])
        contestants.append(data)
    print(f'{c} is DONE!')

AUS REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=AUS AUS is DONE!


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8688\666444661.py:10: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(req_imo.text)


BGR REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=BGR BGR is DONE!
CAN REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=CAN CAN is DONE!
CHN REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=CHN CHN is DONE!
GER REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=GER GER is DONE!
USA REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=USA USA is DONE!
JPN REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=JPN JPN is DONE!
KOR REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=KOR KOR is DONE!
ROU REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=ROU ROU is DONE!
IRN REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=IRN IRN is DONE!
IDN REQUEST STATUS: 200 https://www.imo-official.org/country_individual_r.aspx?code=IDN IDN

In [6]:
#We define the schema structure of the table, it is much like creating an ordinary table in SQL

schema = StructType([
                     StructField('name', StringType(), False),
                     StructField('country_code', StringType(), False),
                     StructField('country', StringType(), False),
                     StructField('year', IntegerType(), False),
                     StructField('p1', IntegerType(), True),
                     StructField('p2', IntegerType(), True),
                     StructField('p3', IntegerType(), True),
                     StructField('p4', IntegerType(), True),
                     StructField('p5', IntegerType(), True),
                     StructField('p6', IntegerType(), True),
                     StructField('total', IntegerType(), False),
                     StructField('abs_rank', IntegerType(), False),
                     StructField('rel_rank(%)', FloatType(), False),
                     StructField('prize', StringType(), False)
                     ])

header = ['id', 'name', 'country_code', 'country', 'year',
          'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'total', 
          'abs_rank', 'rel_rank(%)', 'prize']
df = spark.createDataFrame(contestants, schema=schema)

In [7]:
#to show schema of the df table
df.printSchema()

root
 |-- name: string (nullable = false)
 |-- country_code: string (nullable = false)
 |-- country: string (nullable = false)
 |-- year: integer (nullable = false)
 |-- p1: integer (nullable = true)
 |-- p2: integer (nullable = true)
 |-- p3: integer (nullable = true)
 |-- p4: integer (nullable = true)
 |-- p5: integer (nullable = true)
 |-- p6: integer (nullable = true)
 |-- total: integer (nullable = false)
 |-- abs_rank: integer (nullable = false)
 |-- rel_rank(%): float (nullable = false)
 |-- prize: string (nullable = false)



In [8]:
#Here is the peek of table containing the records of IMO contestant results, 
#it provide their names, countries, the points scored for each problems and the prize the received

df.show()

+--------------------+------------+---------+----+---+---+---+---+---+---+-----+--------+-----------+------------------+
|                name|country_code|  country|year| p1| p2| p3| p4| p5| p6|total|abs_rank|rel_rank(%)|             prize|
+--------------------+------------+---------+----+---+---+---+---+---+---+-----+--------+-----------+------------------+
|       William Cheah|         AUS|Australia|2024|  7|  7|  2|  7|  1|  0|   24|     100|      83.72|      Silver medal|
|            Amber Li|         AUS|Australia|2024|  2|  2|  0|  7|  0|  0|   11|     425|      30.26|Honourable mention|
|Xiangyue (Laura) Nan|         AUS|Australia|2024|  7|  7|  1|  7|  7|  1|   30|      29|      95.39|        Gold medal|
|            Alex Qiu|         AUS|Australia|2024|  7|  1|  0|  7|  0|  0|   15|     327|      46.38|Honourable mention|
|           Cloris Xu|         AUS|Australia|2024|  7|  2|  0|  7|  1|  0|   17|     252|      58.72|      Bronze medal|
|             Iris Xu|         A

## Basic informations about participant countries

In [9]:
df.groupBy('country_code')\
  .agg(first("country").alias('country'),
       min('year').alias('first_appearance'),
       countDistinct('year').alias('appearances'),
       count('*').alias('total_contingents'), 
       countDistinct('name').alias('unique_participants'))\
  .orderBy(col('first_appearance').asc(), 
           col('appearances').desc())\
  .show(truncate = False)

+------------+--------------------------+----------------+-----------+-----------------+-------------------+
|country_code|country                   |first_appearance|appearances|total_contingents|unique_participants|
+------------+--------------------------+----------------+-----------+-----------------+-------------------+
|BGR         |Bulgaria                  |1959            |65         |432              |279                |
|ROU         |Romania                   |1959            |65         |428              |237                |
|HUN         |Hungary                   |1959            |64         |420              |272                |
|USA         |United States of America  |1974            |50         |312              |213                |
|GER         |Germany                   |1977            |47         |288              |183                |
|AUS         |Australia                 |1981            |44         |264              |185                |
|CAN         |Canad

## Countries medal tally of all-time IMO

In [10]:
df.groupBy('country_code')\
  .agg(
       first('country').alias('country'),
       count(when(col('prize')\
                  .like('Gold%'), 1)).alias('Gold'),
       count(when(col('prize')\
                  .like('Silver%'), 1)).alias('Silver'),
       count(when(col('prize')\
                  .like('Bronze%'), 1)).alias('Bronze'),
       count(when(col('prize')\
                  .like('Honour%'), 1)).alias('HM'))\
  .orderBy(col('Gold').desc())\
  .show(truncate=False)

+------------+--------------------------+----+------+------+---+
|country_code|country                   |Gold|Silver|Bronze|HM |
+------------+--------------------------+----+------+------+---+
|CHN         |People's Republic of China|185 |37    |6     |0  |
|USA         |United States of America  |151 |120   |30    |1  |
|RUS         |Russian Federation        |106 |62    |12    |0  |
|KOR         |Republic of Korea         |95  |83    |28    |7  |
|HUN         |Hungary                   |88  |174   |116   |14 |
|ROU         |Romania                   |86  |158   |111   |13 |
|BGR         |Bulgaria                  |57  |130   |121   |19 |
|GER         |Germany                   |54  |114   |91    |16 |
|IRN         |Islamic Republic of Iran  |51  |113   |50    |5  |
|JPN         |Japan                     |48  |98    |52    |6  |
|CAN         |Canada                    |42  |71    |97    |21 |
|THA         |Thailand                  |34  |70    |53    |26 |
|AUS         |Australia  

## Some of best performers of all-time IMO

In [11]:
df.groupBy('name')\
  .agg(
       first("country_code").alias("country"), 
       count('*').alias('appearances'),
       max('year').alias('last_year'),
       count(when(col('prize')\
                  .like('Gold%'), 1)).alias('Gold'),
       count(when(col('prize')\
                  .like('Silver%'), 1)).alias('Silver'),
       count(when(col('prize')\
                  .like('Bronze%'), 1)).alias('Bronze'),
       count(when(col('prize')\
                  .like('Honour%'), 1)).alias('HM'))\
  .filter(col('appearances')>3)\
  .orderBy(col('Gold').desc(), col('Silver').desc(), 
           col('Bronze').desc(), col('HM').desc())\
  .show(15, False)

+--------------------+-------+-----------+---------+----+------+------+---+
|name                |country|appearances|last_year|Gold|Silver|Bronze|HM |
+--------------------+-------+-----------+---------+----+------+------+---+
|Zhuo Qun (Alex) Song|CAN    |6          |2015     |5   |0     |1     |0  |
|Lisa Sauermann      |GER    |5          |2011     |4   |1     |0     |0  |
|Nipun Pitimanaaree  |THA    |5          |2013     |4   |1     |0     |0  |
|Christian Reiher    |GER    |5          |2003     |4   |0     |1     |0  |
|Eric Shen           |CAN    |4          |2023     |4   |0     |0     |0  |
|Luke Robitaille     |USA    |4          |2022     |4   |0     |0     |0  |
|Reid Barton         |USA    |4          |2001     |4   |0     |0     |0  |
|Martin Härterich    |GER    |5          |1989     |3   |1     |1     |0  |
|József Pelikán      |HUN    |4          |1966     |3   |1     |0     |0  |
|Ivan Ivanov         |BGR    |4          |1998     |3   |1     |0     |0  |
|Pranjal Sri

## Countries with most participants who got perfect score (42)

In [12]:
df.groupBy('country_code')\
    .agg(first('country').alias('country'),
         count(when(df['total']==42, 1))\
          .alias('perfect_score_participants'))\
    .orderBy(col('perfect_score_participants').desc())\
    .show(truncate = False)

+------------+--------------------------+--------------------------+
|country_code|country                   |perfect_score_participants|
+------------+--------------------------+--------------------------+
|CHN         |People's Republic of China|47                        |
|USA         |United States of America  |25                        |
|ROU         |Romania                   |18                        |
|GER         |Germany                   |13                        |
|RUS         |Russian Federation        |11                        |
|HUN         |Hungary                   |10                        |
|BGR         |Bulgaria                  |8                         |
|KOR         |Republic of Korea         |6                         |
|CAN         |Canada                    |5                         |
|IRN         |Islamic Republic of Iran  |4                         |
|JPN         |Japan                     |4                         |
|AUS         |Australia           

## Countries medal tally of IMO 2024

In [13]:
df.groupBy('country_code')\
  .agg(
       first('country').alias('country'),
       first('year').alias('year'),
       count(when((col('prize').like('Gold%')) 
                  & (col('year')==2024), 1)).alias('Gold'),
       count(when((col('prize').like('Silver%')) 
                  & (col('year')==2024), 1)).alias('Silver'),
       count(when((col('prize').like('Bronze%')) 
                  & (col('year')==2024), 1)).alias('Bronze'),
       count(when((col('prize').like('Honour%')) 
                  & (col('year')==2024), 1)).alias('HM')
       )\
  .orderBy(col('Gold').desc(), col('Silver').desc(), 
           col('Bronze').desc(), col('HM').desc())\
  .show(truncate=False)

+------------+--------------------------+----+----+------+------+---+
|country_code|country                   |year|Gold|Silver|Bronze|HM |
+------------+--------------------------+----+----+------+------+---+
|CHN         |People's Republic of China|2024|5   |1     |0     |0  |
|USA         |United States of America  |2024|5   |1     |0     |0  |
|IND         |India                     |2024|4   |1     |0     |1  |
|KOR         |Republic of Korea         |2024|2   |4     |0     |0  |
|HUN         |Hungary                   |2024|2   |3     |1     |0  |
|JPN         |Japan                     |2024|2   |2     |1     |1  |
|ROU         |Romania                   |2024|1   |4     |1     |0  |
|IRN         |Islamic Republic of Iran  |2024|1   |3     |1     |1  |
|AUS         |Australia                 |2024|1   |1     |2     |2  |
|IDN         |Indonesia                 |2024|1   |0     |3     |2  |
|CAN         |Canada                    |2024|0   |4     |1     |1  |
|BGR         |Bulgar

## Countries overall score for each problems of IMO 2024

In [14]:
df.groupBy('country_code')\
  .agg(
       first('year').alias('year'),
       sum(when(col('year')==2024, df['p1'])).alias('P1'),
       sum(when(col('year')==2024, df['p2'])).alias('P2'),
       sum(when(col('year')==2024, df['p3'])).alias('P3'),
       sum(when(col('year')==2024, df['p4'])).alias('P4'),
       sum(when(col('year')==2024, df['p5'])).alias('P5'),
       sum(when(col('year')==2024, df['p6'])).alias('P6'),
       sum(when(col('year')==2024, 
                df['total'])).alias('2024_team_score'))\
  .filter(col('year')==2024)\
  .orderBy(col('2024_team_score').desc())\
  .show(truncate=False)

+------------+----+---+---+---+---+---+---+---------------+
|country_code|year|P1 |P2 |P3 |P4 |P5 |P6 |2024_team_score|
+------------+----+---+---+---+---+---+---+---------------+
|USA         |2024|42 |41 |19 |40 |35 |15 |192            |
|CHN         |2024|42 |42 |31 |40 |22 |13 |190            |
|KOR         |2024|42 |37 |18 |42 |7  |22 |168            |
|IND         |2024|42 |34 |11 |42 |28 |10 |167            |
|HUN         |2024|42 |37 |16 |36 |23 |1  |155            |
|ROU         |2024|39 |42 |2  |39 |14 |9  |145            |
|JPN         |2024|42 |30 |4  |35 |21 |11 |143            |
|IRN         |2024|40 |30 |2  |42 |16 |7  |137            |
|CAN         |2024|37 |23 |8  |42 |11 |10 |131            |
|BGR         |2024|42 |15 |4  |42 |20 |3  |126            |
|GER         |2024|40 |27 |2  |32 |19 |0  |120            |
|THA         |2024|38 |32 |1  |35 |8  |2  |116            |
|AUS         |2024|37 |21 |3  |42 |9  |1  |113            |
|IDN         |2024|42 |17 |0  |42 |9  |1

## Some of top gold medallist of IMO 2024

In [15]:
df.filter((df['prize'] == 'Gold medal') & 
          (df['year'] == 2024))\
    .select('name', 'country_code', 'year',
            'abs_rank', 'total', 'prize')\
    .orderBy('abs_rank')\
    .show(15, truncate = False)

+------------------------------+------------+----+--------+-----+----------+
|name                          |country_code|year|abs_rank|total|prize     |
+------------------------------+------------+----+--------+-----+----------+
|Haojia Shi                    |CHN         |2024|1       |42   |Gold medal|
|Alexander Wang                |USA         |2024|3       |38   |Gold medal|
|Satoshi Kano                  |JPN         |2024|4       |37   |Gold medal|
|Qiming Xu                     |CHN         |2024|5       |35   |Gold medal|
|Jessica Wan                   |USA         |2024|5       |35   |Gold medal|
|Hyeongjoe Chu                 |KOR         |2024|5       |35   |Gold medal|
|László Bence Simon            |HUN         |2024|5       |35   |Gold medal|
|Adhitya Mangudy Venkata Ganesh|IND         |2024|5       |35   |Gold medal|
|Linus Tang                    |USA         |2024|14      |32   |Gold medal|
|Ananda Bhaduri                |IND         |2024|14      |32   |Gold medal|

## Indonesian team results for last 15 years

In [16]:
df.groupBy('year')\
  .agg(count(when(df['country_code']=='IDN', 1))\
               .alias('IDN_participants'),
       count(when((col('prize').like('Gold%')) 
                  & (df['country_code']=='IDN'), 1)).alias('Gold'),
       count(when((col('prize').like('Silver%')) 
                  & (df['country_code']=='IDN'), 1)).alias('Silver'),
       count(when((col('prize').like('Bronze%')) 
                  & (df['country_code']=='IDN'), 1)).alias('Bronze'),
       count(when((col('prize').like('Honour%')) 
                  & (df['country_code']=='IDN'), 1)).alias('HM')
       )\
  .filter(col('IDN_participants')>0)\
  .orderBy(col('year').desc())\
  .limit(15)\
  .show()

+----+----------------+----+------+------+---+
|year|IDN_participants|Gold|Silver|Bronze| HM|
+----+----------------+----+------+------+---+
|2024|               6|   1|     0|     3|  2|
|2023|               6|   0|     1|     3|  2|
|2022|               6|   0|     1|     4|  1|
|2021|               6|   0|     2|     4|  0|
|2020|               6|   2|     0|     2|  2|
|2019|               6|   1|     4|     1|  0|
|2018|               6|   1|     5|     0|  0|
|2017|               6|   0|     2|     3|  1|
|2016|               6|   0|     3|     3|  0|
|2015|               6|   0|     2|     4|  0|
|2014|               6|   0|     2|     3|  1|
|2013|               6|   1|     1|     4|  0|
|2012|               6|   0|     1|     3|  1|
|2011|               6|   0|     2|     4|  0|
|2010|               6|   0|     1|     4|  1|
+----+----------------+----+------+------+---+



## Indonesian top scorers of all-time IMO

In [17]:
df.select('name', 'country_code', 'year',
          'total', 'prize')\
    .filter((df['country_code']=='IDN'))\
    .orderBy(df['total'].desc())\
    .show(15, truncate = False)

+-----------------------------------+------------+----+-----+------------+
|name                               |country_code|year|total|prize       |
+-----------------------------------+------------+----+-----+------------+
|Stephen Sanjaya                    |IDN         |2013|35   |Gold medal  |
|Aaron Alvarado Kristanto Julistiono|IDN         |2020|33   |Gold medal  |
|Stanve Avrilium Widjaja            |IDN         |2020|33   |Gold medal  |
|Rafael Kristoforus Yanto           |IDN         |2022|32   |Silver medal|
|Kinantan Arya Bagaspati            |IDN         |2019|32   |Gold medal  |
|Gian Cordana Sanjaya               |IDN         |2018|31   |Gold medal  |
|Rafael Kristoforus Yanto           |IDN         |2023|29   |Silver medal|
|Kevin Adi Senjaya                  |IDN         |2024|29   |Gold medal  |
|Farras Mohammad Hibban Faddila     |IDN         |2018|29   |Silver medal|
|Kinantan Arya Bagaspati            |IDN         |2018|29   |Silver medal|
|Alfian Edgar Tjandra    

## Indonesians with most IMO medals

In [18]:
df.groupBy('name')\
  .agg(
       first("country_code").alias("country"), 
       count('*').alias('apps'),
       sum('total').alias('sum_score'),
       max('year').alias('last_year'),
       count(when(col('prize')\
                  .like('Gold%'), 1)).alias('Gold'),
       count(when(col('prize')\
                  .like('Silver%'), 1)).alias('Silver'),
       count(when(col('prize')\
                  .like('Bronze%'), 1)).alias('Bronze'),
       count(when(col('prize')\
                  .like('Honour%'), 1)).alias('HM')
       )\
  .filter((col('apps')>2) & 
          (col('country')=='IDN') )\
  .orderBy(col('Gold').desc(), col('Silver').desc(), 
           col('Bronze').desc(), col('HM').desc(), 
           col('sum_score').desc())\
  .show(20, False)
  

+-----------------------+-------+----+---------+---------+----+------+------+---+
|name                   |country|apps|sum_score|last_year|Gold|Silver|Bronze|HM |
+-----------------------+-------+----+---------+---------+----+------+------+---+
|Kinantan Arya Bagaspati|IDN    |3   |79       |2019     |1   |1     |1     |0  |
|Gian Cordana Sanjaya   |IDN    |3   |70       |2018     |1   |1     |1     |0  |
|Fransisca Susan        |IDN    |3   |67       |2014     |0   |1     |2     |0  |
|Stefanus Lie           |IDN    |3   |59       |2012     |0   |1     |2     |0  |
|Ahmad Zaky             |IDN    |3   |57       |2012     |0   |1     |2     |0  |
|Tobi Moektijono        |IDN    |3   |57       |2012     |0   |0     |3     |0  |
|Farrel Dwireswara Salim|IDN    |3   |53       |2021     |0   |0     |3     |0  |
|Fajar Yuliawan         |IDN    |3   |32       |2003     |0   |0     |2     |0  |
+-----------------------+-------+----+---------+---------+----+------+------+---+

